# PSyKE's demo

Some imports.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from psyke.cart.predictor import CartPredictor

from psyke import Extractor
from psyke.regression.strategy import AdaptiveStrategy
from psyke.regression import Grid, FeatureRanker, HyperCubeExtractor
from psyke.utils.logic import pretty_theory

Import iris dataset separating features and class.

In [2]:
x, y = load_iris(return_X_y=True, as_frame=True)

Rename of the features.

In [3]:
x.columns = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']

Replace integer indices with the corresponding string class.

In [4]:
y = pd.DataFrame(y).replace({"target": {0: 'setosa', 1: 'versicolor', 2: 'virginica'}})
y

,target
0,setosa
1,setosa
2,setosa
3,setosa
4,setosa
...,...
145,virginica
146,virginica
147,virginica
148,virginica


The final dataset:

In [5]:
dataset = x.join(y)
dataset.columns = [*dataset.columns[:-1], 'iris']
dataset

,SepalLength,SepalWidth,PetalLength,PetalWidth,iris
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


Split between train and test set in a reproducible way.

In [6]:
train, test = train_test_split(dataset, test_size=0.5, random_state=0)

We use as predictor a KNN with K = 4 and we train it.

In [7]:
predictor = KNeighborsClassifier(n_neighbors=7)
predictor.fit(train.iloc[:, :-1], train.iloc[:, -1])
predictor.score(test.iloc[:, :-1], test.iloc[:, -1])

0.96

In [8]:
it = Extractor.iter(predictor, min_update=0.07, min_examples=150, threshold=0.1, max_iterations=600, n_points=1)
theory_from_cream = it.extract(train)
print('ITER performance ({} rules):\nAccuracy = {:.2f}\nFidelity = {:.2f}\n'
      .format(it.n_rules, it.accuracy(test), it.accuracy(test, predictor)))
print('ITER extracted rules:\n\n' + pretty_theory(theory_from_cream))

ITER performance (3 rules):
Accuracy = 0.89
Fidelity = 0.93

ITER extracted rules:

iris(PetalLength, PetalWidth, SepalLength, SepalWidth, setosa) :-
    SepalLength in [4.39, 7.90], SepalWidth in [2.19, 4.10], PetalLength in [1.19, 2.69], PetalWidth in [0.09, 2.50].
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, versicolor) :-
    SepalLength in [4.39, 7.90], SepalWidth in [2.19, 4.10], PetalLength in [2.69, 5.09], PetalWidth in [0.09, 2.50].
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, virginica) :-
    SepalLength in [4.39, 7.90], SepalWidth in [2.19, 4.10], PetalLength in [5.09, 6.90], PetalWidth in [0.09, 2.50].


We create an extractor that uses the CART algorithm and we extract prolog rules from our trained KNN.

In [15]:
DTClassifier = DecisionTreeClassifier(max_leaf_nodes=3).fit(train.iloc[:, :-1], predictor.predict(train.iloc[:, :-1]))
cart = Extractor.cart(CartPredictor(DTClassifier))
theory_from_cart = cart.extract(train)
print(f'CART performance ({cart.n_rules} rules):')
print(f'Accuracy = {cart.accuracy(test):.2f}')
print(f'Fidelity = {cart.accuracy(test, predictor):.2f}\n')
print('\nCART extracted rules:\n\n' + pretty_theory(theory_from_cart))

CART performance (3 rules):
Accuracy = 0.40
Fidelity = 0.39



TypeError: can only concatenate str (not "it.unibo.tuprolog.theory.impl.MutableIndexedTheory") to str

We create a GridEx extractor to extract prolog rules from the same KNN.

In [10]:
ranked = FeatureRanker(x.columns).fit(predictor, x).rankings()
gridEx = Extractor.gridex(predictor, Grid(1, AdaptiveStrategy(ranked, [(0.85, 8)])), threshold=.1, min_examples=1)
theory_from_gridEx = gridEx.extract(train)
print('GridEx performance ({} rules):\nAccuracy = {:.2f}\nAccuracy fidelity = {:.2f}\n'
      .format(gridEx.n_rules, gridEx.accuracy(test), gridEx.accuracy(test, predictor)))
print('GridEx extracted rules:\n\n' + pretty_theory(theory_from_gridEx))

GridEx performance (3 rules):
Accuracy = 0.94
Accuracy fidelity = 0.96

GridEx extracted rules:

iris(PetalLength, PetalWidth, SepalLength, SepalWidth, setosa) :-
    PetalLength in [1.19, 1.91].
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, versicolor) :-
    PetalLength in [2.62, 4.76].
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, virginica) :-
    PetalLength in [4.76, 6.90].


In [11]:
gridEx = Extractor.gridex(predictor, Grid(2, AdaptiveStrategy(ranked, [(0.85, 2)])), threshold=.1, min_examples=1)
theory_from_gridEx = gridEx.extract(train)
print('GridEx performance ({} rules):\nAccuracy = {:.2f}\nAccuracy fidelity = {:.2f}\n'
      .format(gridEx.n_rules, gridEx.accuracy(test), gridEx.accuracy(test, predictor)))
print('GridEx extracted rules:\n\n' + pretty_theory(theory_from_gridEx))

gridEx = Extractor.gridex(predictor, Grid(2, AdaptiveStrategy(ranked, [(0.7, 4)])), threshold=.01, min_examples=1)
theory_from_gridEx = gridEx.extract(train)
print('GridEx performance ({} rules):\nAccuracy = {:.2f}\nAccuracy fidelity = {:.2f}\n'
      .format(gridEx.n_rules, gridEx.accuracy(test), gridEx.accuracy(test, predictor)))
print('GridEx extracted rules:\n\n' + pretty_theory(theory_from_gridEx))

GridEx performance (4 rules):
Accuracy = 0.81
Accuracy fidelity = 0.83

GridEx extracted rules:

iris(PetalLength, PetalWidth, SepalLength, SepalWidth, setosa) :-
    PetalLength in [1.19, 2.62].
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, versicolor) :-
    PetalLength in [2.62, 4.05].
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, versicolor) :-
    PetalLength in [4.05, 5.47].
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, virginica) :-
    PetalLength in [5.47, 6.90].
GridEx performance (6 rules):
Accuracy = 0.97
Accuracy fidelity = 0.97

GridEx extracted rules:

iris(PetalLength, PetalWidth, SepalLength, SepalWidth, setosa) :-
    PetalLength in [1.19, 2.62], PetalWidth in [0.09, 0.7].
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, virginica) :-
    PetalLength in [5.47, 6.90], PetalWidth in [1.3, 1.9].
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, virginica) :-
    PetalLength in [4.05, 6.90], PetalWidth in [1.9, 2.50].
iris(PetalLength

We use CReEPy and CREAM cluster-based extractors to perform the extraction.

In [12]:
creepy = Extractor.creepy(predictor, depth=2, error_threshold=0.1, output=HyperCubeExtractor.Target.CLASSIFICATION)
theory_from_creepy = creepy.extract(train)
print('CReEPy performance ({} rules):\nAccuracy = {:.2f}\nFidelity = {:.2f}\n'
      .format(creepy.n_rules, creepy.accuracy(test), creepy.accuracy(test, predictor)))
print('CReEPy extracted rules:\n\n' + pretty_theory(theory_from_creepy))

CReEPy performance (3 rules):
Accuracy = 0.84
Fidelity = 0.89

CReEPy extracted rules:

iris(PetalLength, PetalWidth, SepalLength, SepalWidth, versicolor) :-
    SepalLength in [4.89, 6.70], SepalWidth in [2.19, 3.20], PetalLength in [2.99, 5.00], PetalWidth in [0.99, 1.80].
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, virginica) :-
    SepalLength in [4.89, 7.70], SepalWidth in [2.19, 3.80], PetalLength in [2.99, 6.90], PetalWidth in [0.99, 2.50].
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, setosa) :-
    SepalLength in [4.39, 7.90], SepalWidth in [2.19, 4.10], PetalLength in [1.19, 6.90], PetalWidth in [0.09, 2.50].


In [13]:
cream = Extractor.cream(predictor, depth=2, error_threshold=0.1, output=HyperCubeExtractor.Target.CLASSIFICATION)
theory_from_cream = cream.extract(train)
print('CREAM performance ({} rules):\nAccuracy = {:.2f}\nFidelity = {:.2f}\n'
      .format(cream.n_rules, cream.accuracy(test), cream.accuracy(test, predictor)))
print('CREAM extracted rules:\n\n' + pretty_theory(theory_from_cream))

CREAM performance (3 rules):
Accuracy = 0.84
Fidelity = 0.89

CREAM extracted rules:

iris(PetalLength, PetalWidth, SepalLength, SepalWidth, versicolor) :-
    SepalLength in [4.89, 6.70], SepalWidth in [2.19, 3.20], PetalLength in [2.99, 5.00], PetalWidth in [0.99, 1.80].
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, virginica) :-
    SepalLength in [4.89, 7.70], SepalWidth in [2.19, 3.80], PetalLength in [2.99, 6.90], PetalWidth in [0.99, 2.50].
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, setosa) :-
    SepalLength in [4.39, 7.90], SepalWidth in [2.19, 4.10], PetalLength in [1.19, 6.90], PetalWidth in [0.09, 2.50].
